In [9]:
import ipywidgets as widgets
from IPython.display import display
import xgboost as xgb
import librosa
import parselmouth
import numpy as np
import pandas as pd
from joblib import load
import nolds
import antropy as ant
from pathlib import Path

# Load pre-trained Parkinson's prediction model
pd_model = load('../models/Custom_model.joblib')

# Define the GUI components
file_upload = widgets.FileUpload(accept='.wav', multiple=False)
predict_button = widgets.Button(description="Predict Parkinson's")
output_widget = widgets.Output()

# Display the Title
display(widgets.HTML("<h1>Parkinson's Prediction from Voice</h1>"))

# Display widgets in the notebook
display(file_upload)
display(predict_button)
display(output_widget)

# Define the function to extract features from the audio file
def extract_features(audio_path):
    sound = parselmouth.Sound(audio_path)
    y, sr = librosa.load(audio_path)

    pitch = sound.to_pitch()
    fo = pitch.selected_array['frequency']
    MDVP_Fo = np.mean(fo[fo > 0])  # Mean fundamental frequency
    MDVP_Fhi = np.max(fo)           # Max fundamental frequency
    MDVP_Flo = np.min(fo[fo > 0])   # Min fundamental frequency

    harmonicity = sound.to_harmonicity()
    HNR = harmonicity.values[harmonicity.values != -200].mean()
    NHR = 1 / HNR if HNR != 0 else 0

    RPDE = ant.perm_entropy(fo, normalize=True)
    DFA = nolds.dfa(y)

    spread = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    Spread2 = spread.mean() / (sr / 2)

    D2 = nolds.corr_dim(fo, emb_dim=25)
    PPE = ant.app_entropy(fo)

    return {
        "MDVP:Fo(Hz)": MDVP_Fo,
        "MDVP:Fhi(Hz)": MDVP_Fhi,
        "MDVP:Flo(Hz)": MDVP_Flo,
        "NHR": NHR,
        "HNR": HNR,
        "RPDE": RPDE,
        "DFA": DFA,
        "Spread2": Spread2,
        "D2": D2,
        "PPE": PPE
    }

# Function to handle the prediction when button is clicked
def on_predict_clicked(b):
    with output_widget:
        output_widget.clear_output()

        if file_upload.value:
            # Access the first uploaded file from the tuple
            uploaded_file = file_upload.value[0]
            
            # Get file content (bytes)
            file_content = uploaded_file['content']

            file_path = Path("temp.wav")
            
            # Save the file locally
            with open(file_path, "wb") as f:
                f.write(file_content)

            # Extract features from the file, convert Path to string
            features = extract_features(str(file_path))
            feature_df = pd.DataFrame([features])
            
            # Predict Parkinson's disease
            prediction = pd_model.predict(feature_df)
            prediction_text = "Signs of Parkinson's" if prediction[0] == 1 else "Healthy"
            
            # Display the result
            print(f"Prediction: {prediction_text}")
        else:
            print("Please upload a .wav file")


# Assign the function to button's click event
predict_button.on_click(on_predict_clicked)


HTML(value="<h1>Parkinson's Prediction from Voice</h1>")

FileUpload(value=(), accept='.wav', description='Upload')

Button(description="Predict Parkinson's", style=ButtonStyle())

Output()